<a href="https://colab.research.google.com/github/sunvod/sormani/blob/master/Progetto_v29_FaceDataset_NASNetMobile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
MODEL_VERSION=29

cosine learning rate adaptation
number ephoc = 100

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Run this cell only if you are using Colab with Drive
#!unzip '/content/drive/My Drive/artificial-neural-networks-and-deep-learning-2020.zip'
#!unzip '/content/drive/My Drive/FaceDataset.zip'
#!unzip '/content/drive/My Drive/FaceDatasetBW.zip'
#!unzip '/content/drive/My Drive/FaceDatasetFull.zip'
#!unzip '/content/drive/My Drive/FaceDatasetFullBW.zip'
!unzip '/content/drive/My Drive/FaceDatasetBig.zip'
#!unzip '/content/drive/My Drive/FaceDatasetBigBW.zip'
#!unzip '/content/drive/My Drive/FaceDatasetExtra.zip'

In [ ]:
import os
import tensorflow as tf
import numpy as np
import io
import matplotlib.pyplot as plt
import PIL
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import pandas as pd
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

SEED = 1234
tf.random.set_seed(SEED)  
np.random.seed(SEED)

In [ ]:
#set global

# Batch size
BS = 32

# img shape
IMG_H = 224
IMG_W = 224

NB_CLASSES = 3
N_EPOCH = 100
LEARNING_RATE=5e-4

# Get current working directory
cwd = os.getcwd()

DATASET = 'FaceDatasetBig'
RESULT_DIR = '/content/drive/MyDrive/results/v' + str(MODEL_VERSION)

MAX_WRONG_SAVED = 9

!mkdir -p $RESULT_DIR

In [ ]:
from tensorflow.keras.applications.nasnet import NASNetMobile
from tensorflow.keras.applications.nasnet import preprocess_input, decode_predictions

In [ ]:
apply_data_augmentation = False

# Create training ImageDataGenerator object
if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(horizontal_flip=True,
                                        vertical_flip=True,
                                        samplewise_center=True,
                                        samplewise_std_normalization=True,
                                        validation_split=0.15,
                                        preprocessing_function=preprocess_input)
else:
    train_data_gen = ImageDataGenerator(validation_split=0.15,
                                        preprocessing_function=preprocess_input)

# Create validation and test ImageDataGenerator objects
valid_data_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
import json
dataset_dir = os.path.join(cwd, DATASET)

with open(os.path.join(cwd, DATASET, 'train_gt.json')) as f:
  d = json.load(f)

traindf = pd.DataFrame(d.items())
traindf.rename(columns={0: 'filename', 1: 'class'}, inplace=True)
traindf['class'] = traindf['class'].astype(str)


print(traindf)

#traindf = traindf.sort_values('filenames')

       filename class
0     14985.jpg     0
1     13358.jpg     0
2     10210.jpg     0
3     18202.jpg     0
4     14962.jpg     0
...         ...   ...
5609  14383.jpg     2
5610  11787.jpg     2
5611  15770.jpg     2
5612  10934.jpg     2
5613  10519.jpg     2

[5614 rows x 2 columns]


In [ ]:
# Create generator to read images from dataset directory for testing
# ------------------------------------------------------------------
dataset_dir = os.path.join(cwd, DATASET)

# Training
training_dir = os.path.join(dataset_dir, 'training')
train_gen = train_data_gen.flow_from_dataframe(traindf,
                                               batch_size=BS,
                                               directory=training_dir,
                                               x_col='filename',
                                               y_col="class",
                                               target_size=(IMG_H, IMG_W),
                                               color_mode="rgb",
                                               classes=None,
                                               class_mode="categorical",
                                               shuffle=True,
                                               seed=SEED,
                                               subset="training") 

Found 4772 validated image filenames belonging to 3 classes.


In [ ]:
# Create generator to read images from dataset directory for validation
# ---------------------------------------------------------------------
dataset_dir = os.path.join(cwd, DATASET)

# Training
training_dir = os.path.join(dataset_dir, 'training')
valid_gen = train_data_gen.flow_from_dataframe(traindf,
                                               batch_size=BS,
                                               directory=training_dir,
                                               x_col='filename',
                                               y_col="class",
                                               weight_col=None,
                                               target_size=(IMG_H, IMG_W),
                                               color_mode="rgb",
                                               classes=None,
                                               class_mode="categorical",
                                               shuffle=False,
                                               seed=SEED,
                                               subset="validation")

Found 842 validated image filenames belonging to 3 classes.


In [ ]:
# Create generators to read images from test directory
# -------------------------------------------------------
dataset_dir = os.path.join(cwd, DATASET)

test_data_gen = ImageDataGenerator(rescale=1./255)
test_gen=test_data_gen.flow_from_directory(directory=dataset_dir,
                                          target_size=(IMG_H, IMG_W),
                                          color_mode="rgb",
                                          classes={'test'},
                                          class_mode="categorical",
                                          batch_size=1,
                                          shuffle=False,
                                          seed=SEED)

Found 450 images belonging to 1 classes.


In [ ]:
# Create Dataset objects
# ----------------------

# Training
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, IMG_H, IMG_W, 3], [None, NB_CLASSES]))
train_dataset = train_dataset.repeat()

# Validation
valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, IMG_H, IMG_W, 3], [None, NB_CLASSES]))
valid_dataset = valid_dataset.repeat()


In [ ]:
#for loading a saved model

def load_model(wrong):
  from keras.models import model_from_json
  # load json and create model
  json_file = open(os.path.join(RESULT_DIR, 'model_v' + str(MODEL_VERSION) + '_' + str(wrong) + ".json"), 'r')
  loaded_model_json = json_file.read()
  json_file.close()
  model = model_from_json(loaded_model_json)
  # load weights into new model
  model.load_weights(os.path.join(RESULT_DIR,  'model_v' + str(MODEL_VERSION) + '_' + str(wrong) + ".h5"))
  print("Loaded model from disk")
  return model

In [ ]:
#model = load_model(8)

RESTART FROM HERE

In [ ]:
net = tf.keras.applications.NASNetMobile(weights='imagenet',
                                        include_top=False,
                                        input_shape=(IMG_H, IMG_W, 3))

In [ ]:
#net.summary()

In [ ]:
model = tf.keras.Sequential()
model.add(net)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=1056, activation='relu')) # 1056
model.add(tf.keras.layers.Dense(units=256, activation='relu')) 
model.add(tf.keras.layers.Dense(units=NB_CLASSES, activation='softmax'))

In [ ]:
# Visualize created model as a table
model.summary()

# Visualize initialized weights
#model.weights

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
NASNet (Functional)          (None, 7, 7, 1056)        4269716   
_________________________________________________________________
flatten_4 (Flatten)          (None, 51744)             0         
_________________________________________________________________
dense_12 (Dense)             (None, 1056)              54642720  
_________________________________________________________________
dense_13 (Dense)             (None, 256)               270592    
_________________________________________________________________
dense_14 (Dense)             (None, 3)                 771       
Total params: 59,183,799
Trainable params: 59,147,061
Non-trainable params: 36,738
_________________________________________________________________


In [ ]:
# Optimization params

loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
metrics = ['accuracy']

In [ ]:
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir /content/drive/My\ Drive/keras_models/

In [ ]:
#import correct predictions
import csv
reader = csv.reader(open('/content/drive/MyDrive/results/results_correct.csv', 'r'))
correct = {}
for row in reader:
   k, v = row
   correct[k] = v
  

In [ ]:
directory = '/content/' + DATASET + '/test/'
import os
import cv2

i = 0
imgs = None
filenames_list = []
for file in os.listdir(directory):
  filename = os.fsdecode(file)
  filenames_list.append(filename)
  img = cv2.imread(directory + filename)
  img_cvt = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img_cvt = img_cvt.astype(np.uint8)
  img_cvt = cv2.resize(img_cvt, (IMG_W, IMG_H))
  img_cvt = np.expand_dims(img_cvt, axis=0)
  if(i == 0):
    imgs = img_cvt
  else:
    imgs = np.concatenate((imgs, img_cvt), axis=0)
  i += 1

imgs = imgs / 255.0 

print(imgs.size)
print(imgs.shape)
print(len(filenames_list))

67737600
(450, 224, 224, 3)
450


In [ ]:
import os
from datetime import datetime

def create_csv(results, results_dir='./', wrong=''):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S - ') + wrong + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')

In [ ]:
def save_model(filename, results_dir='./'):
  model_json = model.to_json()
  try:
    os.remove(os.path.join(results_dir, filename + ".json"))
  except OSError:
    pass
  with open(os.path.join(results_dir, filename + ".json"), "w") as json_file:
    json_file.write(model_json)
  try:
    os.remove(os.path.join(results_dir, filename + ".h5"))
  except OSError:
    pass
  model.save_weights(os.path.join(results_dir, filename + ".h5"))
  print("Saved model to disk")

In [ ]:
import os
from datetime import datetime

# from tensorflow.compat.v1 import ConfigProto
# from tensorflow.compat.v1 import InteractiveSession

# config = ConfigProto()
# config.gpu_options.allow_growth = True
# session = InteractiveSession(config=config)

CWD = '/content/drive/My Drive/keras_models'

if not os.path.exists(CWD):
    os.makedirs(exps_dir)

exp_dir = os.path.join(CWD, 'CNN_n_' + str(datetime.now().strftime('%b%d_%H-%M-%S')))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

# Model checkpoint

ckpt_dir = os.path.join(CWD, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

# Save best weight
ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(CWD, 'cp_{epoch:02d}-{val_loss:.2f}.ckpt'),
                                                   save_weights_only=True,
                                                   monitor='val_loss',
                                                   mode='min',
                                                   save_best_only=True)


# Visualize Learning on Tensorboard
# ---------------------------------
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=1)  # if 1 shows weights histograms
callbacks.append(tb_callback)

# Early Stopping
# --------------
early_stop = False
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                   patience=10,
                                                   verbose=1)
    callbacks.append(es_callback)

# Reduce On Plateau
reduce_on_plateau = True
if reduce_on_plateau:
  rp_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                     factor=0.3,
                                                     patience=5,
                                                     min_lr=0.000145,
                                                     verbose=1)
  #callbacks.append(rp_callback)



In [ ]:
import math

def scheduler(epoch, lr):
  if epoch < 0:
    return lr
  else:
    new_lr = 0.5 * LEARNING_RATE * (1 + math.cos (epoch * math.pi / N_EPOCH))
  print("New learning rate: ", new_lr)
  return new_lr

cosine_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

callbacks.append(cosine_callback)

In [ ]:
import os
from datetime import datetime

class CustomCallback(keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):
      predict = model.predict(x=imgs,
                              steps=len(test_gen),
                              verbose=1)
      i = 0
      n = 1
      results = {}
      wrong = 0
      for el in predict:
        filename = filenames_list[i]
        i += 1
        result = el.argmax()
        if int(correct[filename]) != result:
          wrong += 1
        results[filename] = result
      print('Wrong predictions: ', wrong)
      if(wrong <= MAX_WRONG_SAVED):
        create_csv(results, RESULT_DIR, wrong=str(wrong))
        save_model('model_v' + str(MODEL_VERSION) + '_' + str(wrong) + '-' + str(datetime.now().strftime('%b%d_%H-%M-%S')), results_dir=RESULT_DIR)
  

callbacks.append(CustomCallback())

In [ ]:
history = model.fit(x=train_dataset,
                    epochs=N_EPOCH, 
                    steps_per_epoch=len(train_gen),
                    validation_data=valid_dataset,
                    validation_steps=len(valid_gen), 
                    callbacks=callbacks)


New learning rate:  0.0005
Epoch 1/100
450/450 [==============================] - 9s 20ms/step
Wrong predictions:  240
150/150 [==============================] - 63s 419ms/step - loss: 0.7427 - accuracy: 0.8185 - val_loss: 5.0636 - val_accuracy: 0.3789
New learning rate:  0.0004998766400914329
Epoch 2/100
450/450 [==============================] - 9s 20ms/step
Wrong predictions:  160
150/150 [==============================] - 56s 374ms/step - loss: 0.1493 - accuracy: 0.9466 - val_loss: 6.8139 - val_accuracy: 0.3420
New learning rate:  0.0004995066821070679
Epoch 3/100
450/450 [==============================] - 9s 21ms/step
Wrong predictions:  194
150/150 [==============================] - 57s 380ms/step - loss: 0.1028 - accuracy: 0.9675 - val_loss: 4.0681 - val_accuracy: 0.3979
New learning rate:  0.00049889049115077
Epoch 4/100
450/450 [==============================] - 9s 21ms/step
Wrong predictions:  165
150/150 [==============================] - 57s 378ms/step - loss: 0.1048 - accu

In [ ]:
model.fit(x=train_dataset,
          epochs=500, 
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen), 
          callbacks=callbacks)

In [ ]:
model.fit(x=train_dataset,
          epochs=500, 
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen), 
          callbacks=callbacks)


In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("/content/drive/MyDrive/models/model_v15.json", "w") as json_file:
  json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/content/drive/MyDrive/models/model_v15.h5")
print("Saved model to disk")

Saved model to disk


In [ ]:
# For testing => display some imeges
i = 0
l = 10*0
for el in imgs:
  img = PIL.Image.fromarray((el * 255).astype(np.uint8))
  if i >= l and i <= l + 5:
    display(img)
  i += 1
  if(i >= l + 60):
    break

In [ ]:
# model.load_weights('/path/to/checkpoint')  # use this if you want to restore saved model

predict = model.predict(x=imgs,
                          steps=len(test_gen),
                          verbose=1)


450/450 [==============================] - 7s 16ms/step
